In [8]:
import pandas as pd
import seaborn as sns
import numpy as np
import datetime as dt

In [9]:
df_1 = pd.read_pickle('boxscores_advanced_player_part1.pkl').copy().sort_values('GAME_ID')

In [10]:
df_2 = pd.read_pickle('boxscores_advanced_player_part2.pkl').copy().sort_values('GAME_ID')

In [11]:
df = pd.concat([df_1, df_2], axis=0)
df = df.drop(['START_POSITION', 'COMMENT'], axis=1)
df = df.dropna()
df['MIN'] = (pd.to_datetime(df['MIN'], format="%M:%S", errors='coerce', exact=True).dt.second/60)+pd.to_datetime(df['MIN'], format="%M:%S", errors='coerce', exact=True).dt.minute

In [12]:
df_raw = pd.read_pickle('raw_games_5yrs.pkl').copy()[['TEAM_ID', 'HOME_TEAM', 'GAME_ID', 'GAME_DATE', 'PLUS_MINUS', 'PTS']]

In [13]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13700 entries, 0 to 430
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   TEAM_ID     13700 non-null  int64         
 1   HOME_TEAM   13700 non-null  int64         
 2   GAME_ID     13700 non-null  int32         
 3   GAME_DATE   13700 non-null  datetime64[ns]
 4   PLUS_MINUS  13700 non-null  float64       
 5   PTS         13700 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3)
memory usage: 695.7 KB


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135804 entries, 28 to 20
Data columns (total 30 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   GAME_ID            135804 non-null  object 
 1   TEAM_ID            135804 non-null  int64  
 2   TEAM_ABBREVIATION  135804 non-null  object 
 3   TEAM_CITY          135804 non-null  object 
 4   PLAYER_ID          135804 non-null  int64  
 5   PLAYER_NAME        135804 non-null  object 
 6   NICKNAME           135804 non-null  object 
 7   MIN                133228 non-null  float64
 8   E_OFF_RATING       135804 non-null  float64
 9   OFF_RATING         135804 non-null  float64
 10  E_DEF_RATING       135804 non-null  float64
 11  DEF_RATING         135804 non-null  float64
 12  E_NET_RATING       135804 non-null  float64
 13  NET_RATING         135804 non-null  float64
 14  AST_PCT            135804 non-null  float64
 15  AST_TOV            135804 non-null  float64
 16  AST_R

In [15]:
df['GAME_ID'] = df['GAME_ID'].astype('int32')

In [16]:
df = pd.merge(df, df_raw, on=['TEAM_ID','GAME_ID'], how='inner')

In [17]:
df_sorted = df.sort_values('GAME_DATE')

In [18]:
target_x = ['PLAYER_ID', 'GAME_DATE','MIN', 'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'PACE', 'PIE', 'POSS', 'PTS', 'PLUS_MINUS']

In [19]:
df_rolling = df_sorted[target_x].groupby('PLAYER_ID', group_keys=False).rolling(10, on='GAME_DATE').mean()

In [83]:
scaler = MinMaxScaler()
df_rolling_preproc = scaler.fit_transform(df_rolling)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>)

In [ ]:
df_rolling.to_pickle('/backend/data/pkl/

In [20]:
df_eda = df_rolling.dropna().reset_index()

In [21]:
df_eda

,PLAYER_ID,level_1,GAME_DATE,AST_PCT,AST_RATIO,AST_TOV,DEF_RATING,DREB_PCT,EFG_PCT,MIN,...,OREB_PCT,PACE,PIE,PLUS_MINUS,POSS,PTS,REB_PCT,TM_TOV_PCT,TS_PCT,USG_PCT
0,1713,60811,2018-12-03,0.0764,10.32,0.300,107.47,0.1337,0.3761,14.933333,...,0.0145,108.572,0.0389,-14.9,33.4,102.8,0.0759,3.41,0.4079,0.1783
1,1713,61080,2018-12-05,0.0821,8.99,0.300,103.91,0.1362,0.4047,16.470000,...,0.0185,107.053,0.0490,-14.8,36.6,103.6,0.0767,3.41,0.4404,0.1818
2,1713,61560,2018-12-08,0.0912,9.82,0.400,104.18,0.1434,0.4364,16.000000,...,0.0241,107.495,0.0599,-13.9,35.7,103.6,0.0827,4.24,0.4721,0.1971
3,1713,62243,2018-12-12,0.0779,9.95,0.100,104.79,0.1371,0.3878,15.701667,...,0.0188,108.473,0.0482,-13.9,35.3,104.0,0.0766,3.33,0.4235,0.1908
4,1713,62409,2018-12-14,0.0779,9.95,0.100,109.41,0.1371,0.3711,15.670000,...,0.0138,108.093,0.0473,-11.5,35.2,105.5,0.0734,3.33,0.3994,0.1968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100079,1631323,46417,2023-02-23,0.0574,10.18,0.500,106.21,0.0754,0.3889,11.743333,...,0.0384,106.716,0.0040,0.6,26.0,120.1,0.0568,12.61,0.3981,0.1830
100080,1631323,46797,2023-02-25,0.0665,11.61,0.500,107.64,0.0754,0.4014,12.263333,...,0.0440,106.887,0.0311,2.3,27.2,120.2,0.0604,10.11,0.4309,0.1742
100081,1631323,47239,2023-02-28,0.0798,13.28,0.567,113.30,0.0504,0.3514,13.951667,...,0.0440,106.682,0.0036,-0.8,30.9,117.0,0.0461,12.61,0.3809,0.1517
100082,1631323,47675,2023-03-03,0.0865,13.91,0.667,111.30,0.0245,0.2597,16.165000,...,0.0277,104.909,0.0044,-5.3,35.4,115.3,0.0267,6.57,0.2890,0.1494


In [25]:
df_name = df[['PLAYER_ID', 'PLAYER_NAME']].drop_duplicates('PLAYER_ID')

In [26]:
df_eda = pd.merge(df_eda, df_name, on='PLAYER_ID')

In [34]:
df_eda

,PLAYER_ID,level_1,GAME_DATE,AST_PCT,AST_RATIO,AST_TOV,DEF_RATING,DREB_PCT,EFG_PCT,MIN,...,PACE,PIE,PLUS_MINUS,POSS,PTS,REB_PCT,TM_TOV_PCT,TS_PCT,USG_PCT,PLAYER_NAME
0,1713,60811,2018-12-03,0.0764,10.32,0.300,107.47,0.1337,0.3761,14.933333,...,108.572,0.0389,-14.9,33.4,102.8,0.0759,3.41,0.4079,0.1783,Vince Carter
1,1713,61080,2018-12-05,0.0821,8.99,0.300,103.91,0.1362,0.4047,16.470000,...,107.053,0.0490,-14.8,36.6,103.6,0.0767,3.41,0.4404,0.1818,Vince Carter
2,1713,61560,2018-12-08,0.0912,9.82,0.400,104.18,0.1434,0.4364,16.000000,...,107.495,0.0599,-13.9,35.7,103.6,0.0827,4.24,0.4721,0.1971,Vince Carter
3,1713,62243,2018-12-12,0.0779,9.95,0.100,104.79,0.1371,0.3878,15.701667,...,108.473,0.0482,-13.9,35.3,104.0,0.0766,3.33,0.4235,0.1908,Vince Carter
4,1713,62409,2018-12-14,0.0779,9.95,0.100,109.41,0.1371,0.3711,15.670000,...,108.093,0.0473,-11.5,35.2,105.5,0.0734,3.33,0.3994,0.1968,Vince Carter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100079,1631323,46417,2023-02-23,0.0574,10.18,0.500,106.21,0.0754,0.3889,11.743333,...,106.716,0.0040,0.6,26.0,120.1,0.0568,12.61,0.3981,0.1830,Simone Fontecchio
100080,1631323,46797,2023-02-25,0.0665,11.61,0.500,107.64,0.0754,0.4014,12.263333,...,106.887,0.0311,2.3,27.2,120.2,0.0604,10.11,0.4309,0.1742,Simone Fontecchio
100081,1631323,47239,2023-02-28,0.0798,13.28,0.567,113.30,0.0504,0.3514,13.951667,...,106.682,0.0036,-0.8,30.9,117.0,0.0461,12.61,0.3809,0.1517,Simone Fontecchio
100082,1631323,47675,2023-03-03,0.0865,13.91,0.667,111.30,0.0245,0.2597,16.165000,...,104.909,0.0044,-5.3,35.4,115.3,0.0267,6.57,0.2890,0.1494,Simone Fontecchio


In [77]:
df_vif = df_sorted[['OFF_RATING', 'DEF_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'PACE', 'POSS']].dropna()

In [78]:
df_vif

,OFF_RATING,DEF_RATING,AST_PCT,AST_TOV,AST_RATIO,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,PACE,POSS
57022,96.5,139.7,0.353,3.0,27.3,0.017,9.1,0.357,0.357,0.242,105.75,57
57005,109.5,124.2,0.000,0.0,0.0,0.075,12.5,0.654,0.698,0.239,99.72,63
57004,103.8,128.0,0.111,0.0,16.7,0.143,0.0,0.400,0.400,0.185,99.92,26
57003,101.4,121.6,0.091,2.0,11.1,0.038,5.6,0.667,0.649,0.203,99.80,73
57001,98.1,111.8,0.143,2.0,14.3,0.105,7.1,0.889,0.848,0.207,98.75,52
...,...,...,...,...,...,...,...,...,...,...,...,...
47974,78.9,93.4,0.150,1.0,17.6,0.092,17.6,0.364,0.364,0.177,101.90,76
47975,133.3,77.3,0.000,0.0,0.0,0.000,0.0,0.750,0.750,0.087,107.87,21
47976,96.3,79.5,0.400,8.0,27.6,0.080,3.4,0.553,0.578,0.228,101.11,82
47965,46.2,138.5,0.000,0.0,0.0,0.071,33.3,0.000,0.000,0.200,102.02,13


In [79]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler

In [80]:
scaler = MinMaxScaler()
df_vif_vals = scaler.fit_transform(df_vif)

In [81]:
df_vif_vals

array([[0.27571429, 0.34925   , 0.353     , ..., 0.242     , 0.00734081,
        0.47107438],
       [0.31285714, 0.3105    , 0.        , ..., 0.239     , 0.00692223,
        0.52066116],
       [0.29657143, 0.32      , 0.111     , ..., 0.185     , 0.00693611,
        0.21487603],
       ...,
       [0.27514286, 0.19875   , 0.4       , ..., 0.228     , 0.00701872,
        0.67768595],
       [0.132     , 0.34625   , 0.        , ..., 0.2       , 0.00708189,
        0.10743802],
       [0.41771429, 0.41975   , 0.083     , ..., 0.034     , 0.00698332,
        0.21487603]])

In [82]:
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = df_vif.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df_vif_vals, i)
                          for i in range(len(df_vif.columns))]
  
vif_data

,feature,VIF
0,OFF_RATING,19.508758
1,DEF_RATING,12.543131
2,AST_PCT,6.776613
3,AST_TOV,2.169016
4,AST_RATIO,6.954879
5,REB_PCT,2.648547
6,TM_TOV_PCT,1.660056
7,EFG_PCT,37.861055
8,TS_PCT,45.307898
9,USG_PCT,8.752835
